# Soss implementation of the radon model

Import libraries

In [1]:
using JSON
using Soss, Distributions, NamedTupleTools
using Random
using ArviZ

Check ArviZ python version is up to date, if necessary, update to latest python ArviZ

In [2]:
ArviZ.arviz_version()

v"0.10.0"

In [3]:
# ArviZ.update_arviz()

In [4]:
radon_data = JSON.parsefile("radon.json")
radon_data["county"] = convert(Vector{Int64}, radon_data["county"]) .+ 1
radon_data["county_name"] = convert(Vector{String}, radon_data["county_name"])
radon_data["x"] = convert(Vector{Int64}, radon_data["x"])
radon_data["u"] = convert(Vector{Float64}, radon_data["u"])
radon_data["y"] = convert(Vector{Float64}, radon_data["y"]);

In [5]:
nchains = 4
ndraws = 500;

In [6]:
# Bayesian linear regression.
mod = @model (
    floor_idx, 
    county_idx, 
    uranium, 
    J,
) begin
    # Hyperpriors:
    g ~ Normal(0, 10) |> iid(2)
    sigma_a ~ Exponential(1)
    
    # Varying intercepts uranium model:
    a = g[1] .+ g[2] .* uranium
    za_county ~ Normal(0, 1) |> iid(J)
    a_county = a .+ za_county .* sigma_a
    # Common slope:
    b ~ Normal(0, 1)
    
    # Expected value per county:
    theta = a_county[county_idx] .+ b .* floor_idx
    # Model error:
    sigma ~ Exponential(1)
    
    y ~ For(eachindex(floor_idx)) do i
            Normal(theta[i], sigma)
        end
end

@model (floor_idx, county_idx, uranium, J) begin
        sigma_a ~ Exponential(1)
        sigma ~ Exponential(1)
        g ~ Normal(0, 10) |> iid(2)
        b ~ Normal(0, 1)
        a = g[1] .+ g[2] .* uranium
        za_county ~ Normal(0, 1) |> iid(J)
        a_county = a .+ za_county .* sigma_a
        theta = a_county[county_idx] .+ b .* floor_idx
        y ~ For(eachindex(floor_idx)) do i
                Normal(theta[i], sigma)
            end
    end


In [7]:
constant_data = (
    floor_idx = radon_data["x"], 
    county_idx = radon_data["county"],
    uranium = radon_data["u"],
    J = radon_data["J"],
)
param_mod = mod(; constant_data...);

In [8]:
prior_priorpred = [rand(param_mod, nchains*ndraws)];

In [9]:
post = map(1:nchains) do _
    dynamicHMC(param_mod, (y = radon_data["y"],), ndraws)
end;

In [10]:
pred = predictive(mod, :sigma_a, :sigma, :b, :g, :za_county)
post_postpred = map(post) do post_draws
    map(post_draws) do post_draw
        return merge(post_draw, rand(pred(; constant_data..., post_draw...)))
    end
end;

In [11]:
coords = Dict(
    "level" => ["basement", "floor"],
    "obs_id" => 1:radon_data["N"],
    "county" => radon_data["county_name"],
    "g_coef" => ["intercept", "slope"],
)
dims = Dict(
    "g" => ["g_coef"],
    "za_county" => ["county"],
    "y" => ["obs_id"],
    "floor_idx" => ["obs_id"],
    "county_idx" => ["obs_id"],
    "theta" => ["obs_id"],
    "uranium" => ["county"],
    "a" => ["county"],
    "a_county" => ["county"],
    
)
idata = from_namedtuple(
    post_postpred;
    posterior_predictive = [:y],
    prior = prior_priorpred,
    prior_predictive = [:y],
    coords = coords,
    dims = dims,
    library = Soss,
);

In [12]:
post_data = idata.posterior
theta = post_data.theta.values
sigma = reshape(post_data.sigma.values,size(post_data.sigma.values)...,1)
y = reshape(radon_data["y"],1,1,radon_data["N"])
log_likelihood = logpdf.(Normal.(theta, sigma), y);

In [13]:
idata_aux = from_namedtuple(
    log_likelihood=(y = log_likelihood,), 
    constant_data=constant_data,
    observed_data=(y = radon_data["y"],),
    coords=coords, 
    dims=dims
)
idata = concat(idata, idata_aux)

InferenceData with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [14]:
idata.to_netcdf("soss.nc")

"soss.nc"

## Environment

In [15]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i5-3470S CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, ivybridge)


In [16]:
using Pkg
Pkg.status()

Status `~/.julia/environments/v1.5/Project.toml`
  [131c737c] ArviZ v0.4.7
  [31c24e10] Distributions v0.23.8
  [7073ff75] IJulia v1.21.4
  [682c06a0] JSON v0.21.1
  [d9ec5142] NamedTupleTools v0.13.6
  [438e738f] PyCall v1.92.1
  [8ce77f84] Soss v0.15.3
  [9a3f8284] Random
